In [ ]:
! pip install kaggle

In [3]:
# downloading dataset
import kaggle
import os

path = "artifacts/dataset/"
dataset = "palaksood97/resume-dataset"
if not os.path.exists(path):
    os.makedirs(path)
    kaggle.api.dataset_download_files(dataset= dataset, path= path, unzip= True)
    print(f"Dataset: {dataset} downloaded successfully! ")



ModuleNotFoundError: No module named 'kaggle'

In [1]:
! pip install python-docx

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
# function extracting text from docx format
from docx import Document

def extract_text_from_document(document):
    doc = Document(document)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

text = extract_text_from_document(r"C:\Users\kanis\Mail-It\artifacts\dataset\Resumes\Achyuth Resume_8.docx")
print(text)

ModuleNotFoundError: No module named 'docx'

In [ ]:
! pip install chromadb

In [ ]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")

In [ ]:
def save_to_chromaDB(summarized_text):
    collection.add(
        documents=[summarized_text],
        ids=[str(hash(summarized_text))]
    )

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
! pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile"
)

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    input_variables = ["extracted_text"],
    template = '''You are an AI assistant specialized in extracting and summarizing key information from provided CVs. Given a CV, analyze it and return a structured summary in valid JSON format, capturing only the most essential details.

    CV:
    {extracted_text}

    Return the following output strictly in the JSON format:

        "Full Name": "[Extracted Full Name]",
        "Email": "[Extracted Email Address]",
        "Phone Number": "[Extracted Phone Number]",
        "Location": "[City, Country (if mentioned)]",
        "LinkedIn": "[Extracted LinkedIn Profile URL (if available)]",
        "Portfolio/Website": "[Extracted Portfolio or Website URL (if available)]",
        "Education": [
            
                "Degree": "[Extracted Degree]",
                "University": "[Extracted University Name]",
                "Year of Graduation": "[Year (if available)]"
            
        ],
        "Work Experience": [
            
                "Job Title": "[Extracted Job Title]",
                "Company Name": "[Extracted Company Name]",
                "Location": "[City, Country / Remote]",
                "Employment Type": "[Full-time / Part-time / Contract / Internship]",
                "Duration": "[Start Date - End Date (if available)]",
                "Key Responsibilities": [
                    "Summarized responsibility 1",
                    "Summarized responsibility 2",
                    "Summarized responsibility 3"
                ]
            
        ],
        "Skills": [
            "Extracted skill 1",
            "Extracted skill 2",
            "Extracted skill 3"
        ],
        "Certifications": [
            
                "Certification Name": "[Extracted Certification Name]",
                "Issuing Organization": "[Extracted Organization Name]",
                "Year": "[Year (if available)]"
            
        ],
        "Projects": [
            
                "Project Name": "[Extracted Project Name]",
                "Description": "[Short project summary]",
                "Technologies Used": ["Tech 1", "Tech 2"]
            
        ],
        "Languages": [
            "Extracted language 1",
            "Extracted language 2"
        ],
        "Achievements": [
            "Mentioned achievement 1",
            "Mentioned achievement 2"
        ]

    ### **Guidelines:**
    - Ensure the JSON output is valid and properly formatted.
    - Omit any section if the relevant information is not available in the CV.
    - Keep responses concise and free from unnecessary text.
    - Do not include any additional explanations—only return pure JSON.

    Omit empty data Keep responses concise and free from unnecessary text.There should be no header and text 
    mentioning what you have done, it should just be pure JSON.

    "COMPULSORY"
    Keep the token size of text to be not more than 4000 that is why extract only really important information.
    '''
    )

In [ ]:
from langchain.chains import LLMChain

def cv_summary(extrated_text):
    chain = LLMChain(llm=llm, prompt=prompt_template)
    summarized_text = chain.run(extracted_text=extrated_text)
    return summarized_text

In [ ]:
import os
import time

folder_path = r"C:\Users\kanis\Mail-It\artifacts\dataset\Resumes"

MAX_REQUESTS_PER_MINUTE = 1  # Adjust based on API limits
INTERVAL = 60 / MAX_REQUESTS_PER_MINUTE  # Time gap between requests

i = 0

for file in os.scandir(folder_path):
    if i < 4:  # Process only 10 files
        extracted_text = extract_text_from_document(file)
        summarized_text = cv_summary(extracted_text)
        save_to_chromaDB(summarized_text)

        i += 1
        print(f"Processed {i} files")

        time.sleep(INTERVAL)  # Wait to maintain rate limit
    else:
        break


In [ ]:
job_description = '''About the job
Data Analyst Intern (Paid)

Company: WebBoost Solutions by UM

Location: Remot

Duration: 3 months

Opportunity: Full-time based on performance, with a Certificate of Internship

Application Deadline: 4th April 2025

About WebBoost Solutions by UM

WebBoost Solutions by UM provides students and graduates with hands-on experience in data analysis, helping them enhance their career prospects.

Responsibilities

✅ Collect, clean, and analyze datasets.

✅ Develop reports and data visualizations for business insights.

✅ Identify trends and patterns in data to support decision-making.

✅ Collaborate with teams to present data-driven recommendations.

Requirements

🎓 Enrolled in or a graduate of a relevant program (Data Science, Business Analytics, Computer Science, or related field).

📊 Strong analytical skills and attention to detail.

🐍 Familiarity with tools like Excel, SQL, or Python (preferred).

🤝 Excellent communication and teamwork abilities.

Benefits

💰 Stipend: ₹7,500 - ₹15,000 (Performance-Based) (Paid)

✔ Real-world data analysis experience.

✔ Internship Certificate & Letter of Recommendation.

✔ Build your portfolio with impactful projects.

How to Apply

📩 Submit your application by 4th April 2025 with the subject: "Data Analyst Intern Application".

Equal Opportunity

WebBoost Solutions by UM is an equal-opportunity employer, welcoming applicants from all backgrounds.


'''

In [ ]:
llm_2 = ChatGroq(
    model_name="llama3-8b-8192"
)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["job_description"],
    template='''You are an AI assistant specialized in extracting and summarizing key information from job descriptions. Given a job description, analyze 
    it and return a structured summary in valid JSON format, capturing only the most essential details.

    Job Description:
    {job_description}

    Return the following output strictly in the JSON format:

    "Job Title": "[Extracted Job Title]",
    "Company Name": "[Extracted Company Name (if available)]",
    "Location": "[City, Country / Remote]",
    "Job Type": "[Full-time / Part-time / Contract / Internship]",
    "Key Responsibilities": [
        "Summarized primary duty 1",
        "Summarized primary duty 2",
        "Summarized primary duty 3"
    ],
    "Required Skills": [
        "Extracted technical or soft skill 1",
        "Extracted technical or soft skill 2"
    ],
    "Preferred Qualifications": [
        "Optional qualification 1 (if mentioned)",
        "Optional qualification 2 (if mentioned)"
    ],
    "Experience Required": "[Years of experience (if specified)]",
    "Salary Range": "[Salary details (if mentioned)]",
    "How to Apply": "[Application link or email (if available)]"

    Guidelines:

    Ensure the JSON output is valid and properly formatted.

    Omit any section if the relevant information is not available in the job description.

    Keep responses concise and free from unnecessary text.There should be no header and text 
    mentioning what you have done, it should just be pure JSON.No back ticks.''')



In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)
response = chain.run(job_description=job_description)

In [ ]:
print(response)

{
    "Job Title": "Data Analyst Intern",
    "Company Name": "WebBoost Solutions by UM",
    "Location": "Remote",
    "Job Type": "Internship",
    "Key Responsibilities": [
        "Collect, clean, and analyze datasets",
        "Develop reports and data visualizations for business insights",
        "Identify trends and patterns in data to support decision-making",
        "Collaborate with teams to present data-driven recommendations"
    ],
    "Required Skills": [
        "Strong analytical skills and attention to detail",
        "Excellent communication and teamwork abilities"
    ],
    "Preferred Qualifications": [
        "Familiarity with tools like Excel, SQL, or Python"
    ],
    "Experience Required": "None",
    "Salary Range": "₹7,500 - ₹15,000 (Performance-Based)",
    "How to Apply": "Submit application by 4th April 2025 with the subject: Data Analyst Intern Application"
}


In [ ]:
from langchain.prompts import PromptTemplate

jd_prompt_template = PromptTemplate(
    input_variables=["job_description"],
    template = "Find CVs that match this description: {job_description}"
)

formatted_jd_prompt_template = jd_prompt_template.format(job_description=response)

In [ ]:
# retrieving top 80% match
num_vectors = collection.count()
num_vectors = int(0.8*(num_vectors))
def query(formatted_jd_prompt_template,num_vectors):
    results = collection.query(
    query_texts=[formatted_jd_prompt_template],  
    n_results=num_vectors,  # Retrieve top 5 matches
    include=["documents"]
    )
    documents = results.get("documents", [])  # Get the list of documents
    extracted_data = [{"name": doc["Full Name"], "email": doc["Email"]} for doc in documents]
    return extracted_data
    
result = query(formatted_jd_prompt_template,num_vectors)
result